In [ ]:
import napari
import PIL.Image
import numpy as np

PIL.Image.MAX_IMAGE_PIXELS = 1056323868

v = napari.Viewer()

# lab = np.zeros((109, 2800, 1987), dtype='int')
# lab[10:90,100:2700,100:1900] = 1
# v.add_labels(lab, scale=(100,3.5,3.5))

In [2]:
import zarr
from skimage import io
from skimage import measure
from tqdm import tqdm
import h5py
import numpy as np

## convert ilastik output to labels in tif format

In [ ]:


f = h5py.File(
    "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/191815_05401_CH1_Simple Segmentation.h5",
    "r",
)
pred = f.get("exported_data")
pred = pred[:, :, 0]
mask = pred == 1
mask = mask.astype("int32")
labels = measure.label(mask)
io.imsave("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/54_comps.tif", labels)

## write zarr

In [ ]:
zarr_labels = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='i4')

for z in tqdm(range(52, 57)):
    labels = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/" + str(z) + "_comps.tif")
    zarr_labels[z,:,:] = labels

zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/labels_zarr.zarr', zarr_labels)

In [ ]:
zarr_image = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='i4')

for z in tqdm(range(52, 57)):
    image = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/191815_0" + str(z) + "01_CH1.tif")
    zarr_image[z,:,:] = image

#zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr.zarr', zarr_image)

In [ ]:
v = napari.Viewer()
v.add_image(zarr_image[54,:,:])


In [ ]:
zarr_image.shape

In [ ]:
for x1 in range(0, 20000,2000):
    x2 = np.amin([x1+2000, zarr_image.shape[2]])
    im = zarr_image[52:57,:,x1:x2]
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/" + str(x1) + ".tif"
    io.imsave(fname, im)

In [ ]:
zarr_probs = zarr.zeros((109, 28001, 19868), chunks=(1,1000,1000), dtype='float')
for x1 in range(0, 20000,2000):
    x2 = np.amin([x1+2000, zarr_image.shape[2]])
    f = h5py.File(
        "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/" + str(x1) + "_Probabilities.h5",
        "r",
    )
    pred = f.get("exported_data")
    probs = pred[:,:,:,0]
    zarr_probs[52:57,:,x1:x2] = probs
    break


zarr.save('/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/probs_zarr.zarr', zarr_probs)

## Viterbrain

In [3]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [4]:
sg = state_generation(
    image_path="/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr.zarr",
    ilastik_program_path="/Applications/ilastik-1.4.0b21-OSX.app/Contents/ilastik-release/run_ilastik.sh",
    ilastik_project_path="/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/ilastik/bil_slice.ilp",
    chunk_size=[5,1000,1000],
    soma_coords=[],
    parallel=8,
    prob_path = "/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/probs_zarr.zarr",
)

In [5]:
sg.compute_frags()

Constructing fragment image /Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/image_zarr_labels.zarr of shape (109, 28001, 19868)


/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays

Processing @corner: [0, 0, 3000]
Processing @corner: [0, 0, 5000]
Processing @corner: [0, 0, 4000]
Processing @corner: [0, 0, 1000]
Processing @corner: [0, 0, 0]
Processing @corner: [0, 0, 2000]
Processing @corner: [0, 0, 6000]
Processing @corner: [0, 0, 7000]


looking for components to remove: 0it [00:00, ?it/s]
looking for components to remove: 0it [00:00, ?it/s]
looking for components to remove: 0it [00:00, ?it/s]
looking for components to remove: 0it [00:00, ?it/s]
looking for components to remove: 0it [00:00, ?it/s]
looking for components to remove: 0it [00:00, ?it/s]
looking for components to remove: 0it [00:00, ?it/s]
looking for components to remove: 0it [00:00, ?it/s]


Processing @corner: [0, 0, 8000]


...

looking for components to remove: 0it [00:00, ?it/s]


Processing @corner: [50, 27000, 2000]


looking for components to remove: 0it [00:00, ?it/s]


Processing @corner: [50, 27000, 3000]


looking for components to remove: 0it [00:00, ?it/s]
